### Automação do SUAP-IFPE versão 0.1

#### Importando webdriver e abrindo SUAP

In [59]:
import pandas as pd

arquivo = 'transferencia_palmares.xlsx'
tombos_df = pd.read_excel(arquivo)
#display(tombos_df.head())
# tombos_df = tombos_df.iloc[:5]
print(tombos_df.info())

autentica = pd.read_excel('ls.xlsx')


login = str(autentica['login'][0])
senha = str(autentica['senha'][0])

indisponivel = "indisponivel"

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   tombo   84 non-null     int64
dtypes: int64(1)
memory usage: 800.0 bytes
None


In [8]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
import time



def abrir_suap():
    
    global nav
    nav = webdriver.Chrome()
    nav.get("https://suap.ifpe.edu.br/accounts/login/?next=/")
    # inserindo usuário
    nav.find_element_by_xpath('//*[@id="id_username"]').click()
    nav.find_element_by_xpath('//*[@id="id_username"]').send_keys(login)

    # inserindo senha 
    nav.find_element_by_xpath('//*[@id="id_password"]').click()
    nav.find_element_by_xpath('//*[@id="id_password"]').send_keys(senha)

    nav.find_element_by_xpath('//*[@id="ok"]').click()

In [21]:
def trocar_sala():
    
    abrir_suap()
    
    for linha in tombos_df["tombo"]:
        try:
            nav.get(f"https://suap.ifpe.edu.br/patrimonio/inventario/{linha}/")

            valor_sala_suap = nav.find_element_by_xpath('//*[@id="id_sala"]').get_attribute('value')
            valor_nova_sala = '1990'

            # time.sleep(1)

            if valor_sala_suap != valor_nova_sala:
                nova_sala = Select(nav.find_element_by_id('id_sala'))
                nova_sala.select_by_value(valor_nova_sala)
                time.sleep(1)
                nav.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[2]/input').click()
        
        except NoSuchElementException:
            continue

    
    nav.quit()

In [22]:
def listar_carga():
    
    abrir_suap()
    
    tombo_impressao = []
    carga_impressao = []
    descricao_impressao = []    
    
    for linha in tombos_df["tombo"]:
        
        tombo_impressao.append(linha)
        # print(linha)
        
        try:
            nav.get(f"https://suap.ifpe.edu.br/patrimonio/inventario/{linha}/")

            carga = nav.find_element_by_xpath("/html/body/div[1]/div[3]/table[1]/tbody/tr[7]/td[2]").get_attribute('innerText')
            carga_impressao.append(carga)
            # print(carga)

            descricao = nav.find_element_by_xpath("/html/body/div[1]/div[3]/table[1]/tbody/tr[6]/td[2]/span[1]").get_attribute('innerText')
            descricao_impressao.append(descricao)
            # print(descricao)
        except NoSuchElementException:
            carga_impressao.append(indisponivel)
            descricao_impressao.append(indisponivel)
            continue
            
    tabela_carga = pd.DataFrame()
    tabela_carga['tombo'] = tombo_impressao
    tabela_carga['descricao'] = descricao_impressao
    tabela_carga['carga'] = carga_impressao

    #display(tabela_carga)

    tabela_carga.to_excel("lista_carga"+arquivo, index=False, sheet_name="plan1")
        
    nav.quit()

In [23]:
def impressao_etiqueta():
    
    abrir_suap()
    
    tombo_etiqueta = []    
    empenho_impressao = []
    uasg_impressao = []
    nf_impressao = []
    dtnf_impressao = []
    qrcode_impressao = []
    
    for linha in tombos_df["tombo"]:
        
        tombo_etiqueta.append(linha)
        # print(linha)
        
        try:
            nav.get(f"https://suap.ifpe.edu.br/patrimonio/inventario/{linha}/")
            nav.find_element_by_link_text("Detalhes da Entrada").click()

            empenho = nav.find_element_by_xpath("/html/body/div[1]/div[3]/table[1]/tbody/tr[12]/td/a").get_attribute('innerText')
            empenho = empenho[:12]
            empenho_impressao.append(empenho)
            # print(empenho)
        
            uasg = nav.find_element_by_xpath("/html/body/div[1]/div[3]/table[1]/tbody/tr[1]/td").get_attribute('innerText')
            uasg_impressao.append(uasg)
            # print(uasg)

            nf = nav.find_element_by_xpath("/html/body/div[1]/div[3]/table[1]/tbody/tr[4]/td").get_attribute('innerText')
            nf_impressao.append(nf)
            # print(nf)

            data = nav.find_element_by_xpath("/html/body/div[1]/div[3]/table[1]/tbody/tr[5]/td").get_attribute('innerText')
            dtnf_impressao.append(data)
            # print(data)

            qrcode = str(linha)+" "+empenho+" "+" "+uasg+" "+nf+" "+data
            qrcode_impressao.append(qrcode)

            
        except NoSuchElementException:
            # print(indisponivel)
            empenho_impressao.append(indisponivel)
            uasg_impressao.append(indisponivel)
            nf_impressao.append(indisponivel)
            dtnf_impressao.append(indisponivel)
            qrcode_impressao.append(indisponivel)
            continue    
        
    tabela_impressao = pd.DataFrame()
    tabela_impressao['patrimonio'] = tombo_etiqueta
    tabela_impressao['empenho'] = empenho_impressao
    tabela_impressao['uasg'] = uasg_impressao
    tabela_impressao['nota fiscal'] = nf_impressao
    tabela_impressao['data nota fiscal'] = dtnf_impressao
    tabela_impressao['qrcode'] = qrcode_impressao
    
#     display(tabela_impressao)
    
    tabela_impressao.to_excel("impressao_"+arquivo, index=False, sheet_name="plan1")
        

    nav.quit()

In [61]:
def trocar_carga_contabil():
    
    abrir_suap()
    i = 1
    for linha in tombos_df["tombo"]:
        
        try:
            nav.get(f"https://suap.ifpe.edu.br/patrimonio/inventario/{linha}/")

            nav.find_element_by_xpath('/html/body/div[1]/div[3]/table[1]/tbody/tr[8]/td[2]/select').click()
            nav.find_element_by_xpath('/html/body/div[1]/div[3]/table[1]/tbody/tr[8]/td[2]/select').send_keys('PALM')
            nav.find_element_by_xpath('/html/body/div[1]/div[3]/table[1]/tbody/tr[8]/td[2]/select').send_keys(Keys.ENTER)
            time.sleep(2)
            nav.find_element_by_xpath('/html/body/div[1]/div[3]/table[1]/tbody/tr[8]/td[2]/span').click()
#             nav.find_element_by_link_text("Alterar").click()
            time.sleep(2)    
            
            local = nav.find_element_by_xpath('/html/body/div[1]/div[3]/table[1]/tbody/tr[8]/td[2]').get_attribute('innerText')
#             print(f'tombo: {linha} - carga contabil: {local} - {i}')
            i+=1
            
        except NoSuchElementException:
            local = nav.find_element_by_xpath('/html/body/div[1]/div[3]/table[1]/tbody/tr[8]/td[2]').get_attribute('innerText')
            print(f'tombo: {linha} - carga contabil: {local} - {i}')
            i+=1
            continue

    
    nav.quit()

In [60]:
trocar_carga_contabil()

tombo: 261413 - carga contabil: PALMARES - 0
tombo: 261414 - carga contabil: PALMARES - 1
tombo: 261415 - carga contabil: PALMARES - 2
tombo: 261416 - carga contabil: PALMARES - 3
tombo: 261417 - carga contabil: PALMARES - 4
tombo: 261418 - carga contabil: PALMARES - 5
tombo: 261419 - carga contabil: PALMARES - 6
tombo: 261420 - carga contabil: PALMARES - 7
tombo: 261421 - carga contabil: PALMARES - 8
tombo: 261422 - carga contabil: PALMARES - 9
tombo: 261423 - carga contabil: PALMARES - 10
tombo: 261424 - carga contabil: PALMARES - 11
tombo: 261425 - carga contabil: PALMARES - 12
tombo: 261426 - carga contabil: PALMARES - 13
tombo: 261427 - carga contabil: PALMARES - 14
tombo: 261428 - carga contabil: PALMARES - 15
tombo: 261429 - carga contabil: PALMARES - 16
tombo: 261430 - carga contabil: PALMARES - 17
tombo: 261431 - carga contabil: PALMARES - 18
tombo: 261432 - carga contabil: PALMARES - 19
tombo: 261433 - carga contabil: PALMARES - 20
tombo: 261434 - carga contabil: PALMARES - 2